In [1]:
import torch
import torch.nn.functional as F
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


: 

In [2]:

parquet_file = '/n/holystore01/LABS/iaifi_lab/Lab/nswood/TopLandscape/test_file.parquet'
# config_file = 'data/QuarkGluon/qg_kinpid.yaml'
print('Starting reading')

# Load data from Parquet file
df = pd.read_parquet(parquet_file)
# print(df.columns)
print('Finished reading')
jet_pt, jet_eta,jet_phi, jet_label = df['jet_pt'], df['jet_eta'],df['jet_phi'],df['label']
four_momentum = df[['part_energy', 'part_px', 'part_py', 'part_pz']]
part_eta = df['part_deta']
part_phi = df['part_dphi']

Starting reading


: 

: 

In [4]:
jet_pt, jet_eta,jet_phi, jet_label = df['jet_pt'], df['jet_eta'],df['jet_phi'],df['label']
four_momentum = df[['part_energy', 'part_px', 'part_py', 'part_pz']]
part_eta = df['part_deta']
part_phi = df['part_dphi']

In [129]:

def delta_hyp(dismat):
    p = 0
    row = dismat[p, :][None, :]
    col = dismat[:, p][:, None]
    XY_p = 0.5 * (row + col - dismat)
    maxmin = torch.minimum(XY_p[:, :, None], XY_p[None, :, :]).max(1).values
    return (maxmin - XY_p).max()

In [128]:
import random

def find_neighbors(four_momentum_tensor, dists, index=None, r_percent = 0.05):
    # Randomly select a point
    num_points = four_momentum_tensor.shape[0]
    if index is None:
        index = random.randint(0, num_points - 1)
    selected_point = four_momentum_tensor[index]
    r = r_percent *selected_point[0]
    # Find neighbors within distance r
    neighbors_indices = torch.where(dists[index] <= r)[0]

    neighbors = four_momentum_tensor[neighbors_indices]

    return selected_point, neighbors, neighbors_indices



In [113]:
import random

def find_neighbors_knn(four_momentum_tensor, dists, index=None, k=5):
    # Randomly select a point
    num_points = four_momentum_tensor.shape[0]
    if index is None:
        index = random.randint(0, num_points - 1)
    selected_point = four_momentum_tensor[index]
    
    # Find the indices of the k-nearest neighbors
    neighbors_indices = torch.topk(dists[index], k, largest=False).indices

    neighbors = four_momentum_tensor[neighbors_indices]

    return selected_point, neighbors, neighbors_indices


In [130]:
results = []
for i in range(1000):
    cur_data = four_momentum.iloc[i]
    cur_jet_data = [jet_pt[i], jet_eta[i], jet_label[i]]

    # Extract the columns from the DataFrame
    part_energy = cur_data['part_energy']
    part_px = cur_data['part_px']
    part_py = cur_data['part_py']
    part_pt = np.sqrt(part_px**2 + part_py**2)
    n_parts = len(part_pt)


    # Stack the columns to form an nx4 numpy array
    four_momentum_np = np.stack((part_energy, part_eta.iloc[i] , part_phi.iloc[i]), axis=-1)

    # Convert the numpy array to a torch tensor
    four_momentum_tensor = torch.tensor(four_momentum_np)

    # Extract energy, eta, phi
    energies = four_momentum_tensor[:, 0]
    # normalized_energies = energies / energies.sum()
    # energies = normalized_energies

    # print(normalized_energies)
    etas = four_momentum_tensor[:, 1]
    phis = four_momentum_tensor[:, 2]

    # Compute pairwise energy differences |E_i - E_j|
    energy_diffs = torch.abs(energies[:, None] - energies[None, :])

    # Compute pairwise angular distances ΔR_ij = sqrt((η_i - η_j)^2 + (φ_i - φ_j)^2)
    delta_eta = etas[:, None] - etas[None, :]
    delta_phi = phis[:, None] - phis[None, :]

    # Ensure phi differences wrap around correctly (handle 2pi periodicity)
    delta_phi = torch.remainder(delta_phi + np.pi, 2 * np.pi) - np.pi

    # Calculate ΔR
    delta_R = torch.sqrt(delta_eta**2 + delta_phi**2)

    # Calculate EMD-inspired distance: E_diff * ΔR / R (where R is a scale factor, e.g., jet radius)
    R = 1  # Example jet radius
    dists = (energy_diffs * delta_R) / R
    

    for j in range(len(four_momentum_tensor)):
        for k in [3, 5, 7]:
            selected_point, neighbors, neighbors_indices = find_neighbors_knn(four_momentum_tensor, dists, index=j, k=k)

            # Extract the submatrix for the neighbors
            neighbors_dists = dists[neighbors_indices][:, neighbors_indices]

            delta = delta_hyp(neighbors_dists)
            diam = neighbors_dists.max()
            rel_delta = (2 * delta) / diam

            # Calculate c based on relative delta mean
            c = (0.144 / rel_delta) ** 2

            # Save the results
            results.append({
                'selected_point_energy': selected_point[0].item(),
                'selected_point_eta': selected_point[1].item(),
                'selected_point_phi': selected_point[2].item(),
                'k': k,
                'delta': delta.item(),
                'rel_delta': rel_delta.item(),
                'c': c.item(),
            })

    # Convert the results to a DataFrame
results_df = pd.DataFrame(results)

    selected_point_energy  selected_point_eta  selected_point_phi  k  \
0              218.364243            0.019442            0.093275  3   
1              218.364243            0.019442            0.093275  5   
2              218.364243            0.019442            0.093275  7   
3              153.661118           -0.005887           -0.034971  3   
4              153.661118           -0.005887           -0.034971  5   
5              153.661118           -0.005887           -0.034971  7   
6               76.708054           -0.027067           -0.022574  3   
7               76.708054           -0.027067           -0.022574  5   
8               76.708054           -0.027067           -0.022574  7   
9               64.181671           -0.038218           -0.023472  3   
10              64.181671           -0.038218           -0.023472  5   
11              64.181671           -0.038218           -0.023472  7   
12              57.973267            0.005336           -0.03501

In [132]:
plt.figure(figsize = (10,8))
results_df[results_df['k']==7].plot.scatter(x='selected_point_energy', y='delta', c='k', colormap='viridis')

NameError: name 'plt' is not defined

: 

In [121]:
results = []

for i in range(len(four_momentum_tensor)):
    for k in [3, 5, 7]:
        selected_point, neighbors, neighbors_indices = find_neighbors_knn(four_momentum_tensor, dists, index=i, k=k)

        # Extract the submatrix for the neighbors
        neighbors_dists = dists[neighbors_indices][:, neighbors_indices]

        delta = delta_hyp(neighbors_dists)
        diam = neighbors_dists.max()
        rel_delta = (2 * delta) / diam

        # Calculate c based on relative delta mean
        c = (0.144 / rel_delta) ** 2

        # Save the results
        results.append({
            'selected_point_energy': selected_point[0].item(),
            'selected_point_eta': selected_point[1].item(),
            'selected_point_phi': selected_point[2].item(),
            'k': k,
            'delta': delta.item(),
            'rel_delta': rel_delta.item(),
            'c': c.item(),
        })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)
print(results_df)

     selected_point_energy  selected_point_eta  selected_point_phi  k  \
0               122.238762            0.031814            0.029309  3   
1               122.238762            0.031814            0.029309  5   
2               122.238762            0.031814            0.029309  7   
3               121.227135            0.031344           -0.067194  3   
4               121.227135            0.031344           -0.067194  5   
..                     ...                 ...                 ... ..   
178               0.572157            0.180467           -0.565983  5   
179               0.572157            0.180467           -0.565983  7   
180               0.398239           -0.769833           -0.071215  3   
181               0.398239           -0.769833           -0.071215  5   
182               0.398239           -0.769833           -0.071215  7   

        delta  rel_delta         c  
0    4.885179   0.947752  0.023085  
1    4.885179   0.884551  0.026502  
2    4.88517